In [ ]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import matplotlib.pyplot as plt
 
%matplotlib inline
 
plt.rcParams['figure.figsize']=(20,10)
plt.style.use('ggplot')

In [ ]:
data = pd.read_csv('../input/MRDS_Index_Data_D_Labs_Training_Updated.csv')

In [ ]:
data = data.dropna()
data.tail()

In [ ]:
ds = data[['Date','Ham_Index']]
ds1 = data[['Date','Ham_Index']]

In [ ]:
#ds_reg = data.iloc[:,0:38]
ds_reg=data.copy()

In [ ]:
ds_reg = ds_reg.set_index('Date')

In [ ]:
ds_reg.head()

In [ ]:
ds_reg.drop(['Beef_ME_Imp_cwt','OP_Fuel_Oil_Index','OP_Soyabean_Index','OP_Corn_Index','Pork_CS_Slaughter_Cnt','WR_AvG_Temp','Extreme_Draught_Value','Ham_Index'],axis=1,inplace=True)

In [ ]:
ds=ds.rename(columns={'Date':'ds', 'Ham_Index':'y'})

In [ ]:
ds = ds.set_index('ds')

In [ ]:
ds = ds_reg.join(ds,how='outer')
ds.head()

In [ ]:
ds=ds.reset_index()

In [ ]:
ds=ds.rename(columns={'index':'ds'})

In [ ]:
ds.set_index('ds').y.plot()

In [ ]:
for col in ds.iloc[:,1:]:
    ds[col]=np.log(ds[col])

In [ ]:
#ds['y'] = np.log(ds['y'])
#ds['Pork_Chops_Index'] = np.log(ds['Pork_Chops_Index'])
#ds['Bacon_Index'] = np.log(ds['Bacon_Index'])


In [ ]:
#ds['Beef_CS_Slaughter_Cnt'] = data['Beef_CS_Slaughter_Cnt']

In [ ]:
model = Prophet(changepoint_range=0.98,changepoint_prior_scale=0.5)
for col in ds.iloc[:,1:31]:
    model.add_regressor(col)
model.fit(ds);

In [ ]:
future = model.make_future_dataframe(periods=12, freq = 'm')
future.tail()

In [ ]:
data['Pork_Chops_Index'][-12:].mean()

In [ ]:
for col in ds.iloc[:,1:31]:
    future[col]=ds[col]

In [ ]:
for col in future.iloc[125:137,1:31]:
    future[col][125:137]=ds[col][-12:].mean()

In [ ]:
future[-12:]

In [ ]:
#future['Beef_CS_Slaughter_Cnt'] = ds['Beef_CS_Slaughter_Cnt'].mean()
#for col in ds_reg:
 #   future[col]=ds_reg[col].mean()
#future['Bacon_Index'] = data['Bacon_Index']
#future['Pork_Chops_Index'] = data['Pork_Chops_Index']


#future['Bacon_Index'][0:125] = data['Bacon_Index']
#future['Pork_Chops_Index'][0:125] = data['Pork_Chops_Index']


#future['Bacon_Index'][125:137] = data['Bacon_Index'][-12:].mean()
#future['Pork_Chops_Index'][125:137] = data['Pork_Chops_Index'][-12:].mean()

In [ ]:
future.loc[125:136]['ds']=pd.to_datetime(future.loc[125:136]['ds']).apply(pd.DateOffset(1))
#future.loc[125:136]['ds']
#future.loc[125:136]['Bacon_Index'] = data.loc[-12:]['Bacon_Index'].mean()

In [ ]:
#future['Pork_Chops_Index'] = np.log(future['Pork_Chops_Index'])
#future['Bacon_Index'] = np.log(future['Bacon_Index'])

In [ ]:
forecast = model.predict(future)

In [ ]:
forecast.tail()

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
model.plot(forecast);

In [ ]:
ds.set_index('ds', inplace=True)
forecast.set_index('ds', inplace=True)

In [ ]:
ds1 = ds1.set_index('Date')

In [ ]:
viz_df = ds1.join(forecast[['yhat', 'yhat_lower','yhat_upper']], how = 'outer')

In [ ]:
viz_df.head()

In [ ]:
viz_df['yhat_rescaled'] = np.exp(viz_df['yhat'])
#viz_df['y'] = np.exp(viz_df['y'])

In [ ]:
viz_df.head()

In [ ]:
viz_df[['Ham_Index', 'yhat_rescaled']].plot()

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(viz_df.Ham_Index[:125], viz_df.yhat_rescaled[:125])

In [ ]:
viz_df[-12:]